In [1]:
import numpy as np
import pandas as pd

In [2]:
open_spaces = pd.read_csv('data/locations/open_spaces.csv')
open_spaces = open_spaces.loc[~open_spaces['Type'].isin(['Cemeteries & Burying Grounds', 'Urban Wilds', 'Open Land', 'Other Open Land'])]
canopy = pd.read_csv('data/locations/open_spaces_with_canopy.csv')
streetlights = pd.read_csv('data/locations/open_spaces_with_streetlights.csv')
nearest_bluebike_stations = pd.read_csv('nearest_bluebike_stations_k_10_d_0.5.csv')
nearest_restaurants = pd.read_csv('nearest_restaurants_k_10_d_0.5.csv')
nearest_food_trucks = pd.read_csv('nearest_food_trucks_k_10_d_0.5.csv')
nearest_farmers_markets = pd.read_csv('nearest_farmers_markets_k_7_d_0.5.csv')
nearest_murals = pd.read_csv('nearest_murals_k_10_d_0.5.csv')
roundtrips = pd.read_csv('roundtrips.csv')
park_assets = pd.read_csv('data/locations/park_assets.csv')

In [3]:
selected = [
    'Boston Common',
    'Chinatown Park',
    'City Hall Plaza',
    'Dewey Square Parks',
    'Fallon Field',
    'Faneuil Square',
    'Olmsted Park',
    'Ronan Park',
    'Ross Playground',
    'Carson Beach'
]

In [4]:
open_spaces = open_spaces.loc[open_spaces['Name'].isin(selected)]
open_spaces

,SPACEID,Name,Type,Acres,Geometry,Avg. Lat,Avg. Long,Lat Range,Long Range
24,25,Boston Common,Parks,45.740950,MULTIPOLYGON (((-71.06325725499994 42.35760757...,42.355043,-71.065667,0.005130,0.007050
40,41,City Hall Plaza,"Malls, Squares & Plazas",6.013970,MULTIPOLYGON (((-71.05865943099997 42.36071787...,42.360502,-71.058934,0.002333,0.002581
121,122,Fallon Field,Athletic Fields,7.512146,MULTIPOLYGON (((-71.13383823799995 42.28732795...,42.287572,-71.135249,0.001399,0.003204
159,160,Ronan Park,Parks,11.241366,MULTIPOLYGON (((-71.06341298199999 42.30565509...,42.304354,-71.062631,0.002601,0.003665
279,280,Faneuil Square,"Malls, Squares & Plazas",0.816789,MULTIPOLYGON (((-71.05648443799998 42.36033046...,42.360045,-71.056464,0.000571,0.001081
307,308,Chinatown Park,"Malls, Squares & Plazas",0.841974,MULTIPOLYGON (((-71.05865915699997 42.35217557...,42.351329,-71.059101,0.002085,0.001535
310,311,Dewey Square Parks,"Malls, Squares & Plazas",2.640449,MULTIPOLYGON (((-71.05163775199998 42.35475411...,42.354103,-71.053536,0.001998,0.003796
423,424,Olmsted Park,Parks,42.825230,MULTIPOLYGON (((-71.11463093299994 42.32455331...,42.325980,-71.115734,0.009777,0.007450
477,478,Carson Beach,"Parkways, Reservations & Beaches",24.646397,MULTIPOLYGON (((-71.03837250899994 42.32975120...,42.326421,-71.041152,0.007558,0.015440
483,484,Ross Playground,Playgrounds,13.124954,MULTIPOLYGON (((-71.11254901299998 42.26544443...,42.264119,-71.112935,0.002650,0.003789


In [5]:
park_assets['Feature List'] = park_assets['Feature(s)'].apply(str.split, sep=', ')
playgrounds = set(park_assets.loc[park_assets['Feature List'].apply(lambda x: 'Playground' in x), 'Park'])
athletic_fields = set(park_assets.loc[park_assets['Feature List'].apply(lambda x: 'Athletic Field' in x), 'Park'])

In [6]:
data = open_spaces.loc[:, 'SPACEID':'Acres']

data['Acres (Scaled)'] = np.clip(np.log10(data['Acres'] + 1) / np.log10(51), 0, 1)

data['Canopy Cover Proportion'] = canopy['Canopy Cover Proportion']

data['Streetlight Proportion'] = streetlights['Streetlight Cover Proportion']

data['Number of Nearby Bluebike Stations'] = nearest_bluebike_stations['num_objs']
data[['Daily Roundtrips', 'Weighted Roundtrip Ratio']] = roundtrips[['Daily Roundtrips', 'Weighted Roundtrip Ratio']]

data['Number of Nearby Bluebike Stations (Scaled)'] = np.log10(data['Number of Nearby Bluebike Stations'] + 1)/np.log10(11)
data['Daily Roundtrips (Scaled)'] = np.clip(np.log10(data['Daily Roundtrips'] + 1) / np.log10(25), 0, 1)
data['Weighted Roundtrip Ratio (Scaled)'] = np.clip(np.log10(data['Weighted Roundtrip Ratio'] + 1) / np.log10(1.25), 0, 1)

data['Number of Nearby Restaurants'] = (nearest_restaurants['num_objs']).astype(int)
data['Near Food Trucks'] = nearest_food_trucks['num_objs'] > 0
data['Near Farmers Markets'] = nearest_farmers_markets['num_objs'] > 0

data['Number of Nearby Restaurants (Scaled)'] = np.log10(data['Number of Nearby Restaurants'] + 1) / np.log10(11)

data['Has Athletic Field'] = data['Name'].isin(athletic_fields)
data['Has Playground'] = data['Name'].isin(playgrounds)

data['Near Murals'] = nearest_murals['num_objs'] > 0

data

,SPACEID,Name,Type,Acres,Acres (Scaled),Canopy Cover Proportion,Streetlight Proportion,Number of Nearby Bluebike Stations,Daily Roundtrips,Weighted Roundtrip Ratio,Number of Nearby Bluebike Stations (Scaled),Daily Roundtrips (Scaled),Weighted Roundtrip Ratio (Scaled),Number of Nearby Restaurants,Near Food Trucks,Near Farmers Markets,Number of Nearby Restaurants (Scaled),Has Athletic Field,Has Playground,Near Murals
24,25,Boston Common,Parks,45.740950,0.977821,0.357298,0.965003,10,24.004219,0.053475,1.000000,1.000000,0.233456,10,False,False,1.000000,True,True,True
40,41,City Hall Plaza,"Malls, Squares & Plazas",6.013970,0.495420,0.129559,0.870738,10,12.525618,0.029164,1.000000,0.809160,0.128825,10,True,False,1.000000,False,True,True
121,122,Fallon Field,Athletic Fields,7.512146,0.544656,0.145290,0.758661,3,0.857143,0.130357,0.578130,0.192315,0.549124,10,False,True,1.000000,True,True,True
159,160,Ronan Park,Parks,11.241366,0.637063,0.221674,0.860189,2,0.840265,0.102920,0.458157,0.189479,0.439005,10,False,False,1.000000,True,True,True
279,280,Faneuil Square,"Malls, Squares & Plazas",0.816789,0.151856,0.013949,1.000000,8,12.150693,0.035247,0.916314,0.800427,0.155235,10,True,False,1.000000,False,False,True
307,308,Chinatown Park,"Malls, Squares & Plazas",0.841974,0.155357,0.512517,1.000000,6,9.386377,0.028464,0.811508,0.727116,0.125777,10,False,False,1.000000,False,True,True
310,311,Dewey Square Parks,"Malls, Squares & Plazas",2.640449,0.328628,0.422465,0.959966,7,8.569620,0.023023,0.867194,0.701672,0.102007,10,False,False,1.000000,False,False,True
423,424,Olmsted Park,Parks,42.825230,0.961439,0.612260,0.297061,2,1.450874,0.039661,0.458157,0.278496,0.174304,10,False,False,1.000000,True,False,True
477,478,Carson Beach,"Parkways, Reservations & Beaches",24.646397,0.825165,0.112254,0.311006,3,5.060880,0.142281,0.578130,0.559778,0.596150,8,False,False,0.916314,False,False,True
483,484,Ross Playground,Playgrounds,13.124954,0.673464,0.177787,0.349196,1,0.105485,0.551181,0.289065,0.031155,1.000000,3,False,False,0.578130,True,True,True


In [7]:
data['Environment Score'] = 0.3 * data['Acres (Scaled)'] + 0.7 * data['Canopy Cover Proportion']
data['Safety Score'] = data['Streetlight Proportion']
data['Activity Score'] = 0.1 * data['Number of Nearby Bluebike Stations (Scaled)'] + 0.8 * data['Daily Roundtrips (Scaled)'] + 0.1 * data['Weighted Roundtrip Ratio (Scaled)']
data['Food Score'] = 0.6 * data['Number of Nearby Restaurants (Scaled)'] + 0.2 * data['Near Food Trucks'] + 0.2 * data['Near Farmers Markets']
data['Play Score'] = 0.5 * data['Has Athletic Field'] + 0.5 * data['Has Playground']
data['Art Score'] = data['Near Murals'].astype(int)
data

,SPACEID,Name,Type,Acres,Acres (Scaled),Canopy Cover Proportion,Streetlight Proportion,Number of Nearby Bluebike Stations,Daily Roundtrips,Weighted Roundtrip Ratio,...,Number of Nearby Restaurants (Scaled),Has Athletic Field,Has Playground,Near Murals,Environment Score,Safety Score,Activity Score,Food Score,Play Score,Art Score
24,25,Boston Common,Parks,45.740950,0.977821,0.357298,0.965003,10,24.004219,0.053475,...,1.000000,True,True,True,0.543455,0.965003,0.923346,0.600000,1.0,1
40,41,City Hall Plaza,"Malls, Squares & Plazas",6.013970,0.495420,0.129559,0.870738,10,12.525618,0.029164,...,1.000000,False,True,True,0.239317,0.870738,0.760211,0.800000,0.5,1
121,122,Fallon Field,Athletic Fields,7.512146,0.544656,0.145290,0.758661,3,0.857143,0.130357,...,1.000000,True,True,True,0.265100,0.758661,0.266578,0.800000,1.0,1
159,160,Ronan Park,Parks,11.241366,0.637063,0.221674,0.860189,2,0.840265,0.102920,...,1.000000,True,True,True,0.346291,0.860189,0.241300,0.600000,1.0,1
279,280,Faneuil Square,"Malls, Squares & Plazas",0.816789,0.151856,0.013949,1.000000,8,12.150693,0.035247,...,1.000000,False,False,True,0.055321,1.000000,0.747496,0.800000,0.0,1
307,308,Chinatown Park,"Malls, Squares & Plazas",0.841974,0.155357,0.512517,1.000000,6,9.386377,0.028464,...,1.000000,False,True,True,0.405369,1.000000,0.675421,0.600000,0.5,1
310,311,Dewey Square Parks,"Malls, Squares & Plazas",2.640449,0.328628,0.422465,0.959966,7,8.569620,0.023023,...,1.000000,False,False,True,0.394314,0.959966,0.658257,0.600000,0.0,1
423,424,Olmsted Park,Parks,42.825230,0.961439,0.612260,0.297061,2,1.450874,0.039661,...,1.000000,True,False,True,0.717014,0.297061,0.286043,0.600000,0.5,1
477,478,Carson Beach,"Parkways, Reservations & Beaches",24.646397,0.825165,0.112254,0.311006,3,5.060880,0.142281,...,0.916314,False,False,True,0.326127,0.311006,0.565250,0.549788,0.0,1
483,484,Ross Playground,Playgrounds,13.124954,0.673464,0.177787,0.349196,1,0.105485,0.551181,...,0.578130,True,True,True,0.326490,0.349196,0.153831,0.346878,1.0,1


In [32]:
data['Image'] = data['Name'].apply(lambda x: f"/src/assets/{''.join(x.split()).replace('/', '-')}.jpeg")
data['Map'] = data['Name'].apply(lambda x: f"../../../maps/{x.replace('/', '-')}.json")
data['Hourly Plot'] = data['Name'].apply(lambda x: f"../../../hourly_plots/{x.replace('/', '-')}.json")
data

,SPACEID,Name,Type,Acres,Acres (Scaled),Canopy Cover Proportion,Streetlight Proportion,Number of Nearby Bluebike Stations,Daily Roundtrips,Weighted Roundtrip Ratio,...,Near Murals,Environment Score,Safety Score,Activity Score,Food Score,Play Score,Art Score,Image,Map,Hourly Plot
24,25,Boston Common,Parks,45.740950,0.977821,0.357298,0.965003,10,24.004219,0.053475,...,True,0.543455,0.965003,0.923346,0.600000,1.0,1,/src/assets/BostonCommon.jpeg,../../../maps/Boston Common.json,../../../hourly_plots/Boston Common.json
40,41,City Hall Plaza,"Malls, Squares & Plazas",6.013970,0.495420,0.129559,0.870738,10,12.525618,0.029164,...,True,0.239317,0.870738,0.760211,0.800000,0.5,1,/src/assets/CityHallPlaza.jpeg,../../../maps/City Hall Plaza.json,../../../hourly_plots/City Hall Plaza.json
121,122,Fallon Field,Athletic Fields,7.512146,0.544656,0.145290,0.758661,3,0.857143,0.130357,...,True,0.265100,0.758661,0.266578,0.800000,1.0,1,/src/assets/FallonField.jpeg,../../../maps/Fallon Field.json,../../../hourly_plots/Fallon Field.json
159,160,Ronan Park,Parks,11.241366,0.637063,0.221674,0.860189,2,0.840265,0.102920,...,True,0.346291,0.860189,0.241300,0.600000,1.0,1,/src/assets/RonanPark.jpeg,../../../maps/Ronan Park.json,../../../hourly_plots/Ronan Park.json
279,280,Faneuil Square,"Malls, Squares & Plazas",0.816789,0.151856,0.013949,1.000000,8,12.150693,0.035247,...,True,0.055321,1.000000,0.747496,0.800000,0.0,1,/src/assets/FaneuilSquare.jpeg,../../../maps/Faneuil Square.json,../../../hourly_plots/Faneuil Square.json
307,308,Chinatown Park,"Malls, Squares & Plazas",0.841974,0.155357,0.512517,1.000000,6,9.386377,0.028464,...,True,0.405369,1.000000,0.675421,0.600000,0.5,1,/src/assets/ChinatownPark.jpeg,../../../maps/Chinatown Park.json,../../../hourly_plots/Chinatown Park.json
310,311,Dewey Square Parks,"Malls, Squares & Plazas",2.640449,0.328628,0.422465,0.959966,7,8.569620,0.023023,...,True,0.394314,0.959966,0.658257,0.600000,0.0,1,/src/assets/DeweySquareParks.jpeg,../../../maps/Dewey Square Parks.json,../../../hourly_plots/Dewey Square Parks.json
423,424,Olmsted Park,Parks,42.825230,0.961439,0.612260,0.297061,2,1.450874,0.039661,...,True,0.717014,0.297061,0.286043,0.600000,0.5,1,/src/assets/OlmstedPark.jpeg,../../../maps/Olmsted Park.json,../../../hourly_plots/Olmsted Park.json
477,478,Carson Beach,"Parkways, Reservations & Beaches",24.646397,0.825165,0.112254,0.311006,3,5.060880,0.142281,...,True,0.326127,0.311006,0.565250,0.549788,0.0,1,/src/assets/CarsonBeach.jpeg,../../../maps/Carson Beach.json,../../../hourly_plots/Carson Beach.json
483,484,Ross Playground,Playgrounds,13.124954,0.673464,0.177787,0.349196,1,0.105485,0.551181,...,True,0.326490,0.349196,0.153831,0.346878,1.0,1,/src/assets/RossPlayground.jpeg,../../../maps/Ross Playground.json,../../../hourly_plots/Ross Playground.json


In [33]:
data.set_index('Name').to_json('final_data_table.json', orient='index')